In [5]:
import subprocess, msprime, tskit, os, statistics, pyslim
import numpy as np
import pandas as pd
import random
# import matplotlib.pyplot as plt
# import scipy.stats as stats
# import pylab
import math
# import rpy2
# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr

# energy = importr("energy")

In [2]:
Nrun = 100 # number of SIS runs
Ne = 6200 
Stop = 11 # the stopping criteria
SeqLen = 5.0e06 # sequence length
MutRate = 1e-9 # mutation rate
RecRate = 0 # recombination rate
Tau = 50
SampleSize = 20

In [3]:
# For writing input files
def WriteInput(ts, Nrun, SampleSize, Ne, Stop, SeqLen, MutRate, RecRate, filename):
    filename_tmp = filename + "1"
    f1 = open(filename_tmp,"w+")
    f1.write("Number of runs = %d\n" % Nrun)
    f1.write("Effective population size = %d\n" % Ne)
    f1.write("Recombination = %.15f\n" % RecRate)
    f1.write("Mutation = %.15f\n" % MutRate)
    f1.write("Length = %f\n" % SeqLen)
    f1.write("Stop = %d\n" % Stop)
    f1.write("#")
    f1.close()
    
    filename_tmp = filename + "2"
    f2 = open(filename_tmp,"w+")
    
    # Get distinct haplotypes
    geno =  ts.genotype_matrix() 
    L = len(geno)
    f2.write("Loci = %d\n" % (L+1))
    SampleSize = ts.get_sample_size()
    f2.write("Genes = %d\n" % SampleSize)
          
    # initialise with the first distinct haplotype
    haps_1 = [0]
    haps_2 = [geno[i][0] for i in range(L)]
    haps = np.hstack((haps_1, haps_2))
    # the number of this haplotype is 1
    #haps.append(1)
    haps = np.hstack((haps, [1]))
    haps = np.array([haps])
    ntype = 1
    order = [0]

    for j in range(1,SampleSize):
        new_1 = [0]
        new_2 = [geno[i][j] for i in range(L)]
        new = np.hstack((new_1, new_2))
        flag = 0 # flag=0: a new distinct haplotype
        for k in range(ntype):
            if(haps[k][:(L+1)].tolist() == new.tolist()):
                haps[k][(L+1)] += 1
                flag = 1
                order.append(j)
                break
        if flag == 0:
            order.append(ntype)
            ntype += 1
            #new.append(1)
            new = np.hstack((new, [1]))
            haps = np.vstack([haps, new]) 
          
    f2.write("Distinct Haplotypes = %d\n" % ntype)
    tot=0
    for i in range(ntype):
        tot = tot+ np.sum(haps[i,:(L+1)])*haps[i,(L+1)]
    stationary = tot/(L*SampleSize)
    #stationary = np.sum(haps)/(L*SampleSize)
    f2.write("Stationary = %.5f\n" % (1-stationary))
    print("stationary = %.5f\n" % (1-stationary))  
    
    f2.write("Haplotypes\n")
    for i in range(haps.shape[0]):
        for j in range(haps.shape[1]):
            if j == L+1:
                f2.write("%d\n" % haps[i][j])
            else:
                f2.write("%d " % haps[i][j])
    
    # Mutation transition rates
    f2.write("\nMutation\n0.0 1.0 0.0 1.0\n")
             
    # Genetic positions
    f2.write("\nPositions\n")
    positions = [0.0]
    for s in ts.sites():
        positions.append(s.position)
    for i in range(len(positions)):
        f2.write("%.2f " %(positions[i]))
             
    f2.write("\n#")
    f2.close()
    order = sorted(range(len(order)), key=lambda k: order[k])
    return order;

In [6]:
tree = pyslim.load("recipe_50gb.trees")

In [7]:
tot = 0
for c in range(Ne*2):
    clade1 = c
    count = 0
    ts = tree.at(100)
    for s in range(Ne*2, Ne*4):
        if ts.is_descendant(s, clade1):
            count = count + 1
    if count>0:
        tot = tot + count
        print("clade %d contribute to %d samples" %(c, count))

clade 39 contribute to 3 samples
clade 137 contribute to 52 samples
clade 153 contribute to 14 samples
clade 192 contribute to 1 samples
clade 235 contribute to 35 samples
clade 258 contribute to 12 samples
clade 278 contribute to 8 samples
clade 287 contribute to 1 samples
clade 296 contribute to 7 samples
clade 447 contribute to 19 samples
clade 487 contribute to 54 samples
clade 513 contribute to 16 samples
clade 521 contribute to 20 samples
clade 589 contribute to 103 samples
clade 612 contribute to 22 samples
clade 618 contribute to 31 samples
clade 642 contribute to 25 samples
clade 666 contribute to 57 samples
clade 688 contribute to 28 samples
clade 692 contribute to 34 samples
clade 738 contribute to 12 samples
clade 757 contribute to 9 samples
clade 765 contribute to 3 samples
clade 788 contribute to 8 samples
clade 847 contribute to 12 samples
clade 855 contribute to 30 samples
clade 990 contribute to 22 samples
clade 992 contribute to 24 samples
clade 1009 contribute to 34 

clade 6266 contribute to 18 samples
clade 6269 contribute to 3 samples
clade 6274 contribute to 59 samples
clade 6277 contribute to 3 samples
clade 6286 contribute to 7 samples
clade 6350 contribute to 35 samples
clade 6354 contribute to 16 samples
clade 6367 contribute to 29 samples
clade 6402 contribute to 3 samples
clade 6409 contribute to 15 samples
clade 6411 contribute to 2 samples
clade 6428 contribute to 23 samples
clade 6491 contribute to 23 samples
clade 6507 contribute to 16 samples
clade 6543 contribute to 17 samples
clade 6554 contribute to 14 samples
clade 6568 contribute to 6 samples
clade 6594 contribute to 13 samples
clade 6598 contribute to 10 samples
clade 6614 contribute to 16 samples
clade 6624 contribute to 28 samples
clade 6642 contribute to 6 samples
clade 6643 contribute to 17 samples
clade 6649 contribute to 159 samples
clade 6660 contribute to 6 samples
clade 6681 contribute to 13 samples
clade 6689 contribute to 38 samples
clade 6698 contribute to 9 samples


clade 11999 contribute to 9 samples
clade 12017 contribute to 30 samples
clade 12026 contribute to 16 samples
clade 12079 contribute to 159 samples
clade 12120 contribute to 8 samples
clade 12170 contribute to 23 samples
clade 12173 contribute to 37 samples
clade 12229 contribute to 23 samples
clade 12312 contribute to 14 samples
clade 12327 contribute to 77 samples
clade 12330 contribute to 3 samples
clade 12387 contribute to 2 samples


In [8]:
roots = [1272] #1272, 39
s1 = []

ts = tree.at(100)
for s in range(Ne*2, Ne*4):
    if ts.is_descendant(s, roots[0]):
        s1.append(s)
cases = s1
cases

[14146, 14623, 14641, 14983, 17896, 19789, 19958, 20118, 23607, 24489]

In [9]:
np.random.seed(19)
rest = [x for x in range(Ne*2, Ne*4) if x not in cases]
control = np.random.choice(rest, size=(SampleSize - len(cases)), replace=False)
control

array([15959, 20439, 23864, 23780, 23564, 13906, 23426, 24470, 12683,
       21548])

In [10]:
tree_re = tree.recapitate(recombination_rate=0, Ne=Ne, random_seed=1)
com = np.hstack([cases, control])
tree_com = tree_re.simplify(samples=com, keep_unary=True)
tree_com = msprime.mutate(tree_com, rate=MutRate, random_seed=1, keep=True)
geno_com = tree_com.genotype_matrix()
geno_com.shape

(475, 20)

In [13]:
filename = "infile"
order_com = WriteInput(tree_com, Nrun, SampleSize, Ne, Stop, SeqLen, MutRate, RecRate, filename)

stationary = 0.58137

